# TP4 : Scripting - Manipulation de fichiers

Un script, c'est un petit programme utilitaire qui peut nous aider dans l'automatisation des tâches informatiques en console.  Il s'agit donc d'une application en ligne de commande.  Les scripts sont souvent écrits en bash, en perl, mais également en Python qui se distingue par sa lisibilité et par la richesse de ses librairies.  

Nous allons voir ici comment construire des scripts simples manipulant des fichiers. Nous verrons comment leur donner une interface utilisateur flexible à l'aide d'options.

Dans les TPs suivants, nous élargirons et complexifierons nos scripts pour manipuler des processus ou des ressources réseaux.  

**A retenir**

A partir de la fin de ce TP, tous les scripts que vous écrirez **devront** contenir : 
- La condition "if \__name__ == '\__main__'"
- Une interface en ligne de commande utilisant les options et avec un mode d'emploi, en utilisant par exemple Argparse. 

### Ressources pour le TP

Vous trouverez sur Moodle une série de fichiers dans l'archive data-files.zip.  Téléchargez-la, dézippez-la et placez les fichiers de données directement **dans le même répertoire** que ce notebook.   


## 1.  Lecture de fichiers

Nous allons commencer par étudier comment lire des fichiers situés dans le répertoire courant.  Nous élargirons l'approche à tous les fichiers du système de fichiers ultérieurement, via la manipulation des chemins d'accès.  

Comme vous le savez, préalablement à la lecture d'un fichier, il faut *ouvrir* ce dernier.  Cela s'effectue avec la fonction built-in[`open`](https://docs.python.org/2/library/functions.html#open), qui renvoie un [objet de type fichier](https://docs.python.org/2/library/stdtypes.html#bltin-file-objects) : 

    file = open('data.txt')
    
Une fois le fichier utilisé, il est important de le refermer pour libérer les ressources correspondantes, via la méthode `close()` : 
   
    file.close()
    
Ces manipulations peuvent donner lieu à des erreurs, qui seront signalées par le lancement d'**exceptions**.  Nous étudierons ce concept plus tard dans le cours, mais la gestion de fichiers est une bonne occasion de déjà s'y confronter.  

Comme point de départ, regardons ce qu'il se passe si on essaie d'ouvrir un fichier qui n'existe pas :


In [1]:
file = open('data.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'data.txt'

Le script plante avec un message d'erreur peu esthétique pour un utilisateur néophyte.  Il serait plus propre soit de pouvoir continuer le programme si c'est possible malgré le problème, soit d'avoir un message d'erreur plus propre.  C'est possible grâce à la gestion des exceptions.  

L'exemple ci-dessous montre comment anticiper la survenue d'un problème au niveau de l'ouverture du fichier. Pour cela, on délimite le bloc susceptible de générer une exception via le mot-clé `try`.  Les clauses `except` permettent de réagir à chaque type d'erreur, ici en générant un message d'erreur approprié.  

Essayez d'exécuter ce code, d'abord sans créer de fichier `data.txt` : Vous devriez voir l'erreur correspondante s'afficher.  

Dans un second temps, créez le fichier `data.txt` dans le répertoire dans lequel se trouve ce notebook, et relancez l'exécution du script.  Que voyez-vous comme output?

N'hésitez pas à chipoter le code pour vous l'approprier et bien comprendre les mécanismes utilisés : Enlevez les try/except, changez le nom des erreurs, ... 



In [3]:
try:
    file = open('data.txt')
    print(file)
    file.close()
except FileNotFoundError:
    print('Fichier introuvable.')
except IOError:
    print('Erreur IO.')

<_io.TextIOWrapper name='data.txt' mode='r' encoding='cp1252'>


A présent, nous allons lire un fichier.  Nous nous contenterons ici de la lecture de fichiers textes.  La méthode la plus simple consiste à appeler la méthode `read()`.  Editez votre fichier `data.txt` pour qu'il ait plusieurs lignes de contenu, puis exécutez le morceau de code ci-dessous.   

In [4]:
try:
    file = open('data.txt')
    print(file.read())
    file.close()
except FileNotFoundError:
    print('Fichier introuvable.')
except IOError:
    print('Erreur IO.')

je sais pas si il faut faire des strings mais en tout cas :
Ã§a me saoule !


Ce code n'est pas efficace, car il lit tout le contenu du fichier d'un seul bloc.  D'une part, ce n'est pas pratique, et d'autre part, cela peut être très gourmand en ressources.  Une première manière d'améliorer cela est de lire le fichier ligne par ligne.  Python nous aide pour cela en fournissant un itérateur sur l'objet fichier, ce qui permet d'utiliser facilement une boucle `for` :  


In [6]:
try:
    file = open('data.txt')
    for line in file : 
        print(line)
    file.close()
except FileNotFoundError:
    print('Fichier introuvable.')
except IOError:
    print('Erreur IO.')

je sais pas si il faut faire des strings mais en tout cas :

Ã§a me saoule !


Cependant, l'affichage des lignes de notre fichier n'est pas optimal puisqu'une ligne vide s'affiche après chaque ligne de texte.  Cela vient du fait que lors de la lecture, chaque ligne possède un caractère de retour à la ligne, et qu'il vient donc s'ajouter à celui qui est automatiquement ajouté par la fonction `print()`.  Nous pouvons éviter cela en supprimant ce caractère de retour à la ligne via la méthode [`rstrip()`](https://docs.python.org/3/library/stdtypes.html#str.rstrip) de la classe String.  

In [5]:
try:
    file = open('data.txt')
    for line in file : 
        print(line.rstrip())
    file.close()
except FileNotFoundError:
    print('Fichier introuvable.')
except IOError:
    print('Erreur IO.')

je sais pas si il faut faire des strings mais en tout cas :
Ã§a me saoule !


Nous avons à présent un script qui nous permet de lire un fichier ligne par ligne grâce à un itérateur sur le fichier.  D'autres méthodes peuvent vous être utiles pour lire des éléments d'un fichier : par exemple, `file.readlines()` qui lit tout le fichier d'un coup et place les lignes dans un tableau (attention aux performances), ou `file.readline()` qui lit une seule ligne.  Vous pouvez également vous déplacer dans un fichier grâce à la méthode `file.seek(position)`.  La lecture d'un nombre spécifique d'octets se fera simplement en ajoutant un paramètre à la méthode de base `file.read(nb_bytes)`.   


Cependant, notre code n'est pas encore optimal au niveau de la gestion des ressources : Si une erreur se produit durant la lecture, le fichier ne sera pas correctement fermé, car la ligne `file.close()` ne sera pas exécutée. Nous pouvons ajouter une ligne qui fermera le fichier en cas d'erreur lors de la lecture : 



In [7]:
try:
    file = open('data.txt')
    for line in file : 
        print(line.rstrip())

    file.close()
except FileNotFoundError:
    print('Fichier introuvable.')
except IOError:
    print('Erreur IO.')
    file.close()


je sais pas si il faut faire des strings mais en tout cas :
Ã§a me saoule !


Pour simplifier cela, Python fournit une syntaxe alternative qui permet de garantir que les ressources soient correctement gérées quoi qu'il se passe : il s'agit du mot-clé `with` qui peut être utilisé dans le cadre de l'ouverture d'un fichier.  Dans ce cas, quoi qu'il se passe, le fichier sera correctement fermé quand nécessaire, et il n'y a plus à s'en préoccuper.  

In [10]:
try:
    with open('data.txt') as file : 
        for line in file : 
            print(line.rstrip())
except FileNotFoundError:
    print('Fichier introuvable.')
except IOError:
    print('Erreur IO.')

je sais pas si il faut faire des strings mais en tout cas :
Ã§a me saoule !


### Exercices

1.1 Soit le fichier students.txt. Il contient, sur chaque ligne, le prénom d'un étudiant, son nom et son résultat à un quiz.  Un étudiant peut passer plusieurs fois le même quiz.  Ecrivez un script qui lit ce fichier et imprime, pour chaque étudiant, la moyenne obtenue sur l'ensemble de ses tentatives au quiz.  N'oubliez pas de traiter les cas d'erreur possibles.  

In [21]:
def moyenne(args):
    somme = 0
    for nombre in args:
        somme+= nombre
    return somme / len(args)
try:
    with open("data-files/students.txt") as file:
        list_students = []
        list_students_moyenne = []
        for line in file:
            name = line.rstrip()[:-3]
            result = []
            print(list_students_moyenne)
            if list_students.count(line.rstrip()[:-3]) ==0 and line != "\n":
                if name not in list_students:
                    list_students.append(name)
                    list_students_moyenne.append({name : [line.rstrip()[-2:]]})
                for i in list_students_moyenne:
                    for u in i:
                        print(u)
                    print(i)
                    print(list_students_moyenne)
except FileNotFoundError:
    print('Fichier introuvable.')
except IOError:
    print('Erreur IO.')

[]
Jules Dupont
{'Jules Dupont': ['10']}
[{'Jules Dupont': ['10']}]
[{'Jules Dupont': ['10']}]
Jules Dupont
{'Jules Dupont': ['10']}
[{'Jules Dupont': ['10']}, {'Arthur Tartempion': ['15']}]
Arthur Tartempion
{'Arthur Tartempion': ['15']}
[{'Jules Dupont': ['10']}, {'Arthur Tartempion': ['15']}]
[{'Jules Dupont': ['10']}, {'Arthur Tartempion': ['15']}]
Jules Dupont
{'Jules Dupont': ['10']}
[{'Jules Dupont': ['10']}, {'Arthur Tartempion': ['15']}, {'Emilie Jolie': ['18']}]
Arthur Tartempion
{'Arthur Tartempion': ['15']}
[{'Jules Dupont': ['10']}, {'Arthur Tartempion': ['15']}, {'Emilie Jolie': ['18']}]
Emilie Jolie
{'Emilie Jolie': ['18']}
[{'Jules Dupont': ['10']}, {'Arthur Tartempion': ['15']}, {'Emilie Jolie': ['18']}]
[{'Jules Dupont': ['10']}, {'Arthur Tartempion': ['15']}, {'Emilie Jolie': ['18']}]
Jules Dupont
{'Jules Dupont': ['10']}
[{'Jules Dupont': ['10']}, {'Arthur Tartempion': ['15']}, {'Emilie Jolie': ['18']}, {'Ernest Jone': [' 8']}]
Arthur Tartempion
{'Arthur Tartempion'

1.2. Soit le fichier temp.csv, reprenant les températures minimales et maximales jour par jour.  Ecrivez un script qui donne la moyenne des températures minimales et la moyenne des températures maximale

Conseil : Si vous avez déjà défini une fonction `moyenne()` à l'exercice précédent, elle reste accessible dans les cellules suivantes.  

1.3. Le module json nous permet de récupérer facilement des données au format JSON et les transformer en dictionnaires Python.  Vous pouvez le charger en utilisant le mot-clé `import`: 

    import json

Ouvrez le fichier swapi.json, puis chargez-en le contenu dans un dictionnaire.  Combien de personnages de Star Wars sont listés dans ce fichier?  Listez-les.  


1.4. Vous venez d'être nommé responsable réseau de votre entreprise.  Votre prédécesseur à ce poste n'a réalisé aucune documentation du réseau.  Vous souhaiteriez disposer d'un listing de toutes les interfaces des routeurs et leurs IPs respectives.  Ecrivez un script qui, sur base de l'output de la commande `show ip interface brief` contenu dans un fichier, crée un dictionnaire contenant le nom de l'interface et l'adresse IP correspondante.  

Exemple : voir fichier router0.cfg


## 2. Ecriture de fichiers

Pour écrire dans un fichier, la procédure est similaire à la lecture : Il faut ouvrir le fichier, écrire ce que l'on souhaite y écrire, et puis le fermer.  La différence se situe à deux niveaux : 

- Il faut explicitement indiquer qu'on souhaite ouvrir le fichier en écriture
- Il faut décider, dans le cas où le fichier existe, s'il faut écraser le contenu existant ou écrire à la suite de ce dernier.  

Par défaut, un fichier s'ouvre en lecture ('r') et en mode texte ('t').  Pour l'ouvrir en écriture, il y a deux options possibles : soit le mode 'w' (write), soit le mode 'a' (append).  Le mode 'w' écrase le contenu existant, tandis que le mode 'a' ajoute le nouveau contenu à la fin.  Le mode 'b' permet l'écriture en mode binaire plutôt qu'en mode texte.  Le mode est spécifié comme paramètre supplémentaire dans l'appel à la fonction `open()`.

Ainsi, si je fais :

    file = open('data.txt', 'w')
    
j'ouvre le fichier `data.txt` pour écriture en mode texte.  Par contre, avec : 

    file = open('binary_data', 'ab')
    
j'ouvre le fichier indiqué en mode binaire, avec écriture à la fin du fichier.  

Notez que si un fichier n'existe pas et qu'on essaie de l'ouvrir en écriture, il sera automatiquement créé.  

Enfin, pour écrire, il suffit d'appeler la fonction `write()`, comme montré ci-dessous.  Remarquez l'utilisation du caractère de retour à la ligne pour pouvoir écrire sur plusieurs lignes.  

### Exercices

2.1. Sur base du fichier students.txt de l'exercice 1.1, on souhaiterait cette fois générer un fichier CSV `resultat.csv` reprenant, pour chaque étudiant, sa moyenne et son nombre de tentative.  

Exemple de résultat : 

    nom,prenom,resultat,nb_tentatives
    Jules,Dupont,13.5,2
    Arthur,Tartempion : 15.0,1
    Emilie,Jolie,15.0,3
    Ernest,Jones,8.0,1


2.2. Sur base du fichier `swapi.json`, classez dans un dictionnaire tous les noms de personnages par genre.  Imprimez ce dictionnaire au format json dans un fichier `gender_sw.json`.  

## 3. Système de fichiers et chemins d'accès

Jusqu'ici, nous avons travaillé uniquement avec des fichiers contenus dans le répertoire courant.  Nous allons à présent naviguer dans le système de fichiers et manipuler des chemins d'accès.  

Pour cela, nous exploiterons le module `os`, et plus spécifiquement son sous-module `os.path`. A vous de vous documenter et de trouver les fonctions et méthodes qui vous permettront de résoudre les exercices.  

### Exercices

3.1. Imprimez le chemin du répertoire courant, puis listez-en le contenu.   

3.2. Vérifiez si le fichier "tartempion.txt" existe dans le répertoire courant.  Même chose pour le fichier swapi.json.  



3.3. Listez tous les fichiers contenus dans le répertoire courant. Imprimez à chaque fois le chemin d'accès complet, et indiquez s'il s'agit d'un répertoire ou d'un fichier.  

## 4. Un premier exécutable

Jusqu'ici, nous avons essentiellement travaillé dans des Notebooks Jupyter, qui se prêtent bien à de petits exercices de TPs.  Nous allons à présent changer d'environnement de développement et utiliser des outils permettant de travailler à des programmes de plus grande échelle.  Dans le cadre de ce cours, nous utiliserons PyCharm. 

### Exercices

4.1. Ouvrez PyCharm, et créez un nouveau projet appelé TP4.  Lisez bien les écrans de configuration. PyCharm vous crée un nouveau `venv`.  De quoi s'agit-il?  En quoi est-ce intéressant? Documentez-vous sur Internet pour bien comprendre ce concept essentiel.   
Prenez également soin de vérifier quel est l'interpréteur Python utilisé.  

4.2. Copiez/coller le code de l'exercice 3.3. (par exemple) dans un nouveau fichier python appelé tp4.py.  Vérifiez qu'il s'exécute dans pyCharm.  

4.3. (sous Unix) Depuis un terminal, naviguez vers la localisation du fichier tp4.py.  Pouvez-vous l'exécutez depuis la ligne de commande?  Ajoutez le "shebang" adéquat et vérifiez les permissions pour transformer ce script en exécutable.  Testez-le en tapant  :

    > ./tp4.py 
    
Si vous travaillez sous Windows, vous pouvez lancer votre programme en donnant le nom du script à l'interpréteur Python. Il est également possible de créer un exécutable, mais cela ne sera pas abordé dans ce TP.  N'hésitez pas à creuser le sujet par vous-même. 

PyCharm vous fournit également un accès console direct, depuis la fenêtre du bas de l'écran. 

4.4. Tout fichier Python peut être considéré comme un module, et être importé depuis un autre script.  Créez un second script appelé par ex. `tp4_bis.py`, et écrivez-y : 
    
    import tp4

Exécutez ce script.  Que se passe-t'il?  Est-ce un comportement souhaitable lorsqu'on importe des éléments?  

Pour éviter cela, nous pouvons préciser à l'interpréteur le comportement à adopter selon qu'on est dans un script directement exécuté, ou dans un module importé depuis un autre fichier.  Cela s'effectue via le code ci-dessous : 

    if __name__ == '__main__':
        <code à exécuter lors d'un appel direct du script>
        

        
### A retenir

A partir de maintenant, dans tous vos fichiers, faites bien la distinction entre les éléments qui peuvent être réutilisés par d'autres modules (fonctions, classes et méthodes) et les éléments destinés à être exécutés directement depuis le script (instructions directement interprétables).  Ces derniers doivent donc figurer dans la condition ci-dessous.  


## 5. Script avec options

Le comportement des utilitaires en ligne de commande peut être ajusté à l'aide des arguments et des options.  Vous connaissez ainsi tous la commande `ls`, et certaines options fréquemment utilisées : 
    
    > ls -lh *.txt

Dans cet exemple, l'utilitaire `ls` utilise un argument (dit "positionnel" car sa position est prise en compte) : le nom des éléments à lister (ici via l'utilisation d'un wildcard \*), et deux options `l` et `h`, repérables au tiret qui les précède.  Ces deux options sont au format court (une seule lettre).  Il existe également des options au format long (mot), qui sont dans ce cas précédées d'un double tiret --.  Les options sont facultatives et leur ordre n'est pas fixé, contrairement aux arguments positionnels.  
    
Ce format pour les arguments et les options est très fréquemment utilisé dans les utilitaires en ligne de commande, et nous allons également le suivre pour personnaliser le comportement de nos script Python.  

Une première approche consiste à analyser les arguments de la ligne de commande au niveau système.  Ces derniers sont disponibles dans la variable `sys.argv`.  

Voici le résultat obtenu si on exécute un `print(sys.argv)` dans un nouveau script tp4_sysargs.py : 

    (venv) (base) ➜  TP4 python tp4_sysargs.py          
    ['tp4_sysargs.py']
    (venv) (base) ➜  TP4 python tp4_sysargs.py arg1
    ['tp4_sysargs.py', 'arg1']
    (venv) (base) ➜  TP4 python tp4_sysargs.py arg1 arg2
    ['tp4_sysargs.py', 'arg1', 'arg2']
    (venv) (base) ➜  TP4 python tp4_sysargs.py --monOption monArgument
    ['tp4_sysargs.py', '--monOption', 'monArgument']

Nous pourrions nous baser là-dessus pour récupérer les options et les paramètres.  Cette méthode est cependant laborieuse puisqu'il faut passer en revue tous les cas de figure possibles au niveau des options : options courtes, options longues, arguments facultatifs, etc.  

Il existe heureusement des librairies qui permettent de gérer les options et les arguments de manière beaucoup plus simple.  Nous verrons ici la librairie Argparse fournie dans la bibliothèque Python, mais il en existe d'autres (Docopt, Click par ex.). 

Nous vous invitons à vous familiariser avec Argparse en lisant le [tutoriel officiel](https://docs.python.org/fr/3/howto/argparse.html#id1) sur le sujet.  

### Exercice

5.1. Nous allons enrichir l'exercice 3.3 pour en faire un utilitaire plus complet. Créez dans pyCharm un script appelé my_utils.py qui liste le contenu d'un répertoire donné.  Ce dernier doit pouvoir être appelé en ligne de commandes suivant la documentation ci-dessous : 

    usage: my_utils.py [-h] [-f] directory

    positional arguments:
      directory             directory whose entries must be listed

    optional arguments:
      -h, --help            show this help message and exit
      -f, --files      list only files and hide subdirectories 
      -l, --long            show full pathname for each entry
      
Vous aurez donc besoin : 
    - d'un argument positionnel obligatoire
    - de deux flags (options booléennes) -f et -l

Implémentez cette solution dans un nouveau script my_utils.py, et testez-le depuis la console PyCharm.  
